In [53]:
import pandas as pd
import glob
import deepl
from dotenv import load_dotenv
import os
load_dotenv() 

# df = pd.read_excel('data/nlmyo/processed/diag.xlsx')
# # Read the text content from the filename column and add it to a new column
# df["filepath"] = df["filename"].apply(lambda x: "data/nlmyo/processed/" + x)
# df['raw_text'] = df['filepath'].apply(lambda x: open(x, 'r').read())


# translator = deepl.Translator(os.getenv("DEEPL_KEY")) 
# df['deepl_translation'] = df['raw_text'].apply(lambda x: translator.translate_text(x, target_lang="EN-US").text)
# save df to csv file
# df.to_csv('data/nlmyo/processed/diag_translated.csv', index=False)

True

In [67]:
import numpy as np
from embetter.external import CohereEncoder, OpenAIEncoder
from embetter.grab import ColumnGrabber
from sklearn.pipeline import make_pipeline 
import pandas as pd
from dotenv import load_dotenv
import os

import openai

# load_dotenv() 

# openai.organization = os.getenv("OPENAI_ORG")
# openai.api_key = os.getenv("OPENAI_KEY")
# # This pipeline grabs the `text` column from a dataframe

# df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')

# text_emb_pipeline = make_pipeline(
#   ColumnGrabber("deepl_translation"),
#   OpenAIEncoder(model="text-embedding-ada-002")
# )

# X = text_emb_pipeline.fit_transform(df, df['diag_simple'])
# np.save('data/nlmyo/processed/report_translated_embed_openai.npy', X)

In [71]:
df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')
df['diag_simple'].value_counts()

Nermaline Myopathy        37
Core Myopathy             37
UNCLEAR                   37
Centronuclear Myopathy    12
NON-MC                    11
CFTD                       2
Name: diag_simple, dtype: int64

In [72]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#### Import the data
df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')
Y = df['diag_simple'].values
X_openai = np.load('data/nlmyo/processed/report_translated_embed_openai.npy')

# Remove CFTD and unclear diagnosis
df['diag_simple'].value_counts()
df['diag_simple'] = df['diag_simple'].replace('CFTD', 'UNCLEAR')
# Drop the rows with unclear diagnosis
df = df[df['diag_simple'] != 'UNCLEAR']
Y = df['diag_simple'].values
# Do the same for the X array based on the df index
X_openai = X_openai[df.index]
cv_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [57]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the medicine document for classification: "
)
text_reports = df["raw_text"].to_list()
embeddings_results = embeddings.embed_documents(text_reports)
X_instructor = np.array(embeddings_results)
X_instructor.shape


load INSTRUCTOR_Transformer
max_seq_length  512


(97, 768)

In [73]:
clf_dummy = DummyClassifier(strategy='prior')
cv_scores_dummy = cross_val_score(clf_dummy, X_openai, Y, cv=cv_fold)
print("Dummy Classifier Results:")
print(f"All CV Scores: {cv_scores_dummy}")
print(f"Mean CV Score:  {np.mean(cv_scores_dummy)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores_dummy)}")

Dummy Classifier Results:
All CV Scores: [0.4        0.4        0.4        0.4        0.3        0.3
 0.3        0.33333333 0.33333333 0.33333333]
Mean CV Score:  0.35000000000000003
Standard Deviation CV Score: 0.04281744192888378


In [74]:
clf = LogisticRegression(max_iter=3000)
cv_scores = cross_val_score(clf, X_openai, Y, cv=cv_fold)
print("Results with Logistic Regression and OpenAI Embeddings on English Translated Reports:")
print(f"All CV Scores: {cv_scores}")
print(f"Mean CV Score:  {np.mean(cv_scores)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores)}")

Results with Logistic Regression and OpenAI Embeddings on English Translated Reports:
All CV Scores: [0.6        0.4        0.6        0.6        0.5        0.5
 0.6        0.77777778 0.44444444 0.66666667]
Mean CV Score:  0.5688888888888889
Standard Deviation CV Score: 0.10515127257174203


In [75]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass


# Create a pipeline
pipe = Pipeline([('clf', DummyEstimator())]) # Placeholder Estimator

# Candidate learning algorithms and their hyperparameters
search_space = [{'clf': [LogisticRegression()],
                    'clf__max_iter': [1500]},
                {'clf': [GaussianNB()],},
                {'clf': [MLPClassifier()],
                 'clf__max_iter': [500]},
                {'clf': [KNeighborsClassifier()],},
                {'clf': [SVC()],},
                {'clf': [GaussianProcessClassifier()],},
                {'clf': [HistGradientBoostingClassifier()],},
                {'clf': [DecisionTreeClassifier()],},
                {'clf': [RandomForestClassifier()],},
                {'clf': [AdaBoostClassifier()],},
                ]


In [76]:
# Create grid search 
gs = GridSearchCV(pipe, search_space, scoring="accuracy", cv=cv_fold)
gs.fit(X_openai, Y)
df_cv_search = pd.DataFrame(gs.cv_results_)
# df_cv_search.to_csv('data/nlmyo/processed/report_translated_embed_cohere_gridsearch.csv')
df_cv_search

/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.045115,0.008719,0.001008,0.000115,LogisticRegression(),1500,"{'clf': LogisticRegression(), 'clf__max_iter':...",0.6,0.4,0.6,0.6,0.5,0.5,0.6,0.777778,0.444444,0.666667,0.568889,0.105151,4
1,0.002982,0.000151,0.001270,0.000058,GaussianNB(),NaN,{'clf': GaussianNB()},0.4,0.4,0.4,0.4,0.3,0.4,0.6,0.888889,0.333333,0.444444,0.456667,0.162272,9
2,2.286303,0.291888,0.001653,0.000499,MLPClassifier(max_iter=500),500,"{'clf': MLPClassifier(max_iter=500), 'clf__max...",0.6,0.8,0.7,0.7,0.8,0.2,0.8,0.777778,0.555556,0.777778,0.671111,0.177375,1
3,0.001050,0.000129,0.012559,0.015670,KNeighborsClassifier(),NaN,{'clf': KNeighborsClassifier()},0.6,0.6,0.3,0.5,0.5,0.3,0.3,0.888889,0.444444,0.444444,0.487778,0.172021,7
4,0.004119,0.000203,0.001852,0.000348,SVC(),NaN,{'clf': SVC()},0.5,0.4,0.5,0.4,0.4,0.4,0.7,0.666667,0.444444,0.555556,0.496667,0.106696,6
5,0.067736,0.004791,0.008938,0.001559,GaussianProcessClassifier(),NaN,{'clf': GaussianProcessClassifier()},0.6,0.4,0.6,0.6,0.5,0.4,0.6,0.666667,0.444444,0.666667,0.547778,0.097885,5
6,1.270725,0.053013,0.014721,0.013036,HistGradientBoostingClassifier(),NaN,{'clf': HistGradientBoostingClassifier()},0.6,0.6,0.5,0.4,0.7,0.2,0.6,0.777778,0.555556,0.777778,0.571111,0.166385,3
7,0.038774,0.004010,0.000779,0.000080,DecisionTreeClassifier(),NaN,{'clf': DecisionTreeClassifier()},0.4,0.6,0.5,0.6,0.2,0.3,0.4,0.444444,0.222222,0.444444,0.411111,0.132031,10
8,0.287393,0.027552,0.016484,0.001058,RandomForestClassifier(),NaN,{'clf': RandomForestClassifier()},0.5,0.6,0.5,0.7,0.7,0.5,0.7,0.777778,0.555556,0.777778,0.631111,0.107474,2
9,0.706422,0.075892,0.010056,0.000553,AdaBoostClassifier(),NaN,{'clf': AdaBoostClassifier()},0.5,0.4,0.6,0.5,0.5,0.4,0.4,0.555556,0.444444,0.333333,0.463333,0.077786,8


In [77]:
gs.best_estimator_

Pipeline(steps=[('clf', MLPClassifier(max_iter=500))])

In [78]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# Create grid search 
cls = RandomForestClassifier(random_state=42)
gs_rf = GridSearchCV(cls, param_grid, scoring="accuracy", cv=cv_fold)
gs_rf.fit(X_openai, Y)
df_cv_search_rf = pd.DataFrame(gs_rf.cv_results_)
#df_cv_search.to_csv('data/nlmyo/processed/report_translated_embed_cohere_gridsearch.csv')
df_cv_search_rf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030950,0.008222,0.002565,0.000454,None,1,2,10,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.6,...,0.4,0.8,0.6,0.4,0.777778,0.555556,0.555556,0.568889,0.136337,131
1,0.126134,0.002270,0.007640,0.000091,None,1,2,50,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.6,...,0.7,0.7,0.6,0.5,0.777778,0.555556,0.555556,0.608889,0.107129,109
2,0.249313,0.003261,0.014936,0.001400,None,1,2,100,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.5,...,0.7,0.7,0.6,0.6,0.888889,0.666667,0.777778,0.673333,0.102824,2
3,0.510782,0.035231,0.030652,0.004715,None,1,2,200,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.6,...,0.7,0.7,0.6,0.7,0.777778,0.666667,0.777778,0.672222,0.067449,17
4,0.027612,0.001927,0.002492,0.000121,None,1,5,10,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.7,...,0.7,0.6,0.6,0.4,0.777778,0.555556,0.666667,0.590000,0.136757,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.451584,0.007384,0.029253,0.002168,20,4,5,200,"{'max_depth': 20, 'min_samples_leaf': 4, 'min_...",0.6,...,0.7,0.6,0.6,0.7,0.888889,0.666667,0.777778,0.673333,0.102824,2
140,0.023827,0.001377,0.002346,0.000075,20,4,10,10,"{'max_depth': 20, 'min_samples_leaf': 4, 'min_...",0.6,...,0.7,0.4,0.4,0.5,0.666667,0.555556,0.555556,0.557778,0.095167,134
141,0.113031,0.002047,0.007736,0.000205,20,4,10,50,"{'max_depth': 20, 'min_samples_leaf': 4, 'min_...",0.5,...,0.7,0.5,0.4,0.6,0.888889,0.666667,0.777778,0.633333,0.137257,68
142,0.222299,0.003643,0.014493,0.000331,20,4,10,100,"{'max_depth': 20, 'min_samples_leaf': 4, 'min_...",0.5,...,0.7,0.5,0.4,0.7,0.888889,0.666667,0.666667,0.622222,0.136897,86


In [80]:
# Print the best parameters and score
print("Best parameters:", gs_rf.best_params_)
print("Best score:", gs_rf.best_score_)


best_rf = RandomForestClassifier(**gs_rf.best_params_, random_state=42)
best_rf.fit(X_openai, Y)

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

# Use cross_val_predict to get predicted labels and probabilities
y_pred = cross_val_predict(best_rf, X_openai, Y, cv=cv_fold)
y_probas = cross_val_predict(best_rf, X_openai, Y, cv=cv_fold, method='predict_proba')
# Compute classification report
report = classification_report(Y, y_pred, target_names=set(Y), output_dict=True)


Best parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best score: 0.6833333333333333


/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/meyer/code-project/AI-dev-playground/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [81]:
# Save the trained model to disk & WandB
import joblib
joblib.dump(best_rf, 'openAI_model.joblib')

# experiment tracking
import wandb

run = wandb.init(project='myo-text-classify',
                 config={"embedding": "openAI", "doc_lang": "fr"})
best_estimator = gs_rf.best_estimator_
config = wandb.config
best_params = gs_rf.best_params_
best_score = gs_rf.best_score_
best_std = gs_rf.cv_results_['std_test_score'][gs_rf.best_index_]


wandb.log({'Classification Report': report,
           'Best Params': best_params,
           'Best Score': best_score,
           'CV Std Devs': best_std},)

wandb.sklearn.plot_classifier(best_estimator, X_openai, X_openai, Y, Y, y_pred, y_probas, labels=list(set(Y)),
                                                         model_name='instructor_model', feature_names=None)
# Create artifact for best model
model_artifact = wandb.Artifact('openAI_model', type='model')
# Add best estimator to artifact
model_artifact.add_file('openAI_model.joblib')
# Log artifact to WandB
wandb.run.log_artifact(model_artifact)
wandb.finish()

wandb: 
wandb: Plotting instructor_model.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: WARNING This function only supports binary classification at the moment and therefore expects labels to be binary. Skipping calibration curve.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


Best Score,▁
CV Std Devs,▁
Best Score,0.68333
CV Std Devs,0.10003
